In [89]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import geom

# The Coinflipping Room

Context: You have a room of n people each with a biased coin that gives heads/success with probability p on a given flip (Bernoulli). Every hour, everyone in the room flips their coin and records their result. What is the expected number of hours until everyone in the room has gotten at-least one heads.

In [90]:
# Const: Number of simulations
NUM_SIM = 100
# Const: Number of trials to simulate, max (should be infinite :P)
NUM_TRIALS = 200

# Number of r.v.s
n = 50
# Success rate per trial
p = 0.05

Attempt 1: Simulate the game, tracking the number of trials vs the # of successful random variables so far

In [91]:
outcomes = [] # Tuples of (trial #, cum count)
for _ in range(0, NUM_SIM):
    remaining_rvs = n
    for i in range(0, NUM_TRIALS):
        successes = sum(np.random.uniform(0, 1, remaining_rvs) < p)
        remaining_rvs -= successes
        outcomes.append((i, (n - remaining_rvs)))
        # Early stop
        if remaining_rvs == 0:
            break
df = pd.DataFrame(outcomes, columns=['num_trials', 'cum_success'])

In [92]:
fig = px.scatter(x=df['num_trials'], y=df['cum_success'])
fig.show()

Attempt 2: This time, let's simulate the number of successful random variables against the expected number of trials.

In [93]:
# To invert the graph, let's take each simulation and see
# how long it took to reach each "checkpoint" between 0 and n
# This way, we can somewhat uniformly simulate how many
# iterations it took to reach every "Num of successes"
outcomes = []

for _ in range(0, NUM_SIM):
    remaining_rvs = n
    outcomes.append((0, 0))  # (Cum successes, trial)
    for i in range(1, NUM_TRIALS + 1):
        past_succ = n - remaining_rvs
        successes = sum(np.random.uniform(0, 1, remaining_rvs) < p)
        remaining_rvs -= successes
        
        
        # With this delta, add every "checkpoint" we have met
        for j in range(past_succ + 1, past_succ + successes + 1):
            outcomes.append((j, i))
        # Early stop
        if remaining_rvs == 0:
            break

df = pd.DataFrame(outcomes, columns=['cum_success', 'num_trials'])


In [94]:
fig = px.scatter(x=df['cum_success'], y=df['num_trials'])
fig.show()

In [95]:
# Slicing at n gives us a distribution over "trials until n successes"
fig = px.histogram(df[df['cum_success'] == n], x='num_trials')
fig.show()

In [96]:
df[df['cum_success'] == n]['num_trials'].mean()

84.39

Attempt 3: A simpler version of the game, just tracking the total number of trials it look for the whole room to succeed

In [97]:
# Just take the number of trials it took to complete
outcomes = []

for _ in range(0, NUM_SIM):
    remaining_rvs = n
    for i in range(1, NUM_TRIALS + 1):
        successes = sum(np.random.uniform(0, 1, remaining_rvs) < p)
        remaining_rvs -= successes
    
        if remaining_rvs == 0:
            outcomes.append(i)
            break

df = pd.DataFrame(outcomes, columns=['num_trials'])


In [98]:
fig = px.histogram(df, x='num_trials')
fig.show()

In [99]:
np.mean(outcomes)

88.41414141414141

Attempt 4: More simply, we now simulate this using the max of geometric variables

In [100]:
from scipy.stats import geom
import plotly.express as px

In [101]:
max_geo_outcomes = []
for _ in range(0, NUM_SIM):
    r = geom.rvs(p, size=n)
    max_geo_outcomes.append(max(r))
df = pd.DataFrame(max_geo_outcomes, columns=["num_trials"])

In [102]:
fig = px.histogram(df, x='num_trials')
fig.show()

In [103]:
np.mean(max_geo_outcomes)

85.93